# Towards a Reliable and Interpretable Approach forBuilding Process Prediction Models

Description: Here

## 0. Inport necessary library & Set workspace

In [ ]:
import os
import numpy as np
import pandas as pd

from natsort import natsorted
from datetime import datetime
from dateutil.relativedelta import relativedelta

import math
import itertools

# library for plotting
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf

from keras import backend as K
from keras.models import Model
from keras.preprocessing import sequence
from keras.utils.data_utils import Sequence
from keras.regularizers import l2
from keras.constraints import non_neg, Constraint
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, confusion_matrix


from keras.layers import Input, Concatenate, Flatten
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adagrad
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

from tensorflow.keras import utils as ku 
from nltk.util import ngrams
import pickle


import keras
print(keras.__version__)


In [ ]:
MY_WORKSPACE_DIR = os.path.join(os.getcwd(),"BPIC_11")
try:
    os.makedirs(MY_WORKSPACE_DIR)
    print("Directory " , MY_WORKSPACE_DIR ,  " created")
except:
    print("Directory " , MY_WORKSPACE_DIR ,  " already exists")

OUTPUT_DIR = os.path.join(MY_WORKSPACE_DIR, "output_files")

try:
    os.makedirs(OUTPUT_DIR)
    print("Directory " , OUTPUT_DIR ,  " created")
except:
    print("Directory " , OUTPUT_DIR ,  " already exists")

In [ ]:


# The args dictionary is adapted from GenerativeLSTM by Manuel Camargo
# https://github.com/AdaptiveBProcess/GenerativeLSTM/blob/master/dg_training.py

args = dict()

args['processed_training_vec'] = os.path.join(OUTPUT_DIR, 'vec_training.p')
args['processed_test_vec'] = os.path.join(OUTPUT_DIR, 'vec_test.p')
args['weights'] = os.path.join(OUTPUT_DIR, 'weights.p')
args['indexes'] = os.path.join(OUTPUT_DIR, 'indexes.p')
args['args'] = os.path.join(OUTPUT_DIR, 'args.p') 

args['url'] = "https://www.win.tue.nl/bpi/lib/exe/fetch.php?media=2011:hospital_log.csv.zip"
args['log_name'] = 'bpic2011_Hospital_Data'
args['file_name'] = os.path.join(MY_WORKSPACE_DIR, 'BPIC_2011.csv') 
args['processed_file_name'] = os.path.join(OUTPUT_DIR, 'BPIC_2011_Processed.csv')
args['task']='outcome'

args['lstm_act'] = None # optimization function see keras doc
args['dense_act'] = None # optimization function see keras doc
args['n_size'] = 15 # n-gram size
args['l_size'] = 50 # LSTM layer sizes
args['norm_method'] = 'lognorm' # max, lognorm

In [ ]:
args

## 1. Data Cleaning

The [raw dataset](https://www.win.tue.nl/bpi/lib/exe/fetch.php?media=2011:hospital_log.csv.zip) is from Business Processing Intelligence Challenge (BPIC) 2011. Find more information [here](https://www.win.tue.nl/bpi/doku.php?id=2011:challenge).

In [ ]:
# Dataframe creation
raw_data = pd.read_csv(args['url'], compression='zip', sep=";", low_memory=False, dtype='str')
raw_data.to_csv(path_or_buf=args['file_name'],sep=';', index=False)

df = raw_data
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

### Delete irrelevant attributes

From the analysis of the columns for bpic11 and the standard of event log, we can divide attributes (columns) into `Trace` attributes and `Event` attributes. The `Trace` attributes share the same values in one case. While, the `Event` attributes keep changing between activities. Here list all the attribiutes for bpic11:

- Attributes for traces (Static)
    - case:concept:name
    - Age, Age:1-15
    - ~~Diagnosis, Diagnosis:1-15~~
    - Diagnosis code, Diagnosis code:1-15
    - Treatment code, Treatment code:1-15
    - ~~Diagnosis Treatment Combination ID, Diagnosis Treatment Combination ID:1-15~~
    - ~~Specialism code, Specialism code:1-15~~
    - ~~Start date, Start date:1-15~~
    - ~~End date, End date:1-15~~

- Attributes for events (Dynamic)
    - org:group
    - Number of executions
    - ~~Specialism code~~
    - event:concept:name
    - Producer code
    - Section
    - Activity code
    - time:timestamp
    - ~~lifecycle:transition~~

All irrelevant attributes with delete line need to be droped.

In [ ]:
# Clean all columns with "Diagnosis", "Diagnosis Treatment Combination ID", "Specialism code" and "lifecycle:transition"
df = df.loc[:,~df.columns.str.startswith('Diagnosis:')]
df = df.drop(columns='Diagnosis')
df = df.loc[:,~df.columns.str.startswith('Specialism code')] 
df = df.loc[:,~df.columns.str.startswith('lifecycle:transition')]
df = df.loc[:,~df.columns.str.startswith('Diagnosis Treatment Combination ID')]

df = df.loc[:,~df.columns.str.startswith('Start date')] 
df = df.loc[:,~df.columns.str.startswith('End date')]

df = df.drop(columns='Unnamed: 127')
df.shape


In [ ]:
df = df.astype({'case:concept:name': 'int64', 'Number of executions': 'int64'})
df.head()

### Combine repeating features to a list

In [ ]:
# Sort all columns in alphabetical order

df = df.reindex(natsorted(df.columns), axis=1)
df.head()

From the previous analysis, only three static `Trace` attributes - `Age`, `Diagnosis code` and `Treatment code` - contain repeating columns. Here we use functions for combining the repearting features into lists.

For three feature lists, we apply different methods to choose a suitable value.

- `Age`: Because the diagnosis and treatment for a patient may be over years, the age in one case may increase. Here we choose the latest one.

In [ ]:
# function for finding last valid value
def findLastValid(x):
    list_of_last_value = []
    for index, row in x.iterrows():
        if row.last_valid_index() is None:
            list_of_last_value.append(np.nan)
        else:
            list_of_last_value.append(row[row.last_valid_index()])
    
    return list_of_last_value

Age_List = findLastValid(df.loc[:, df.columns.str.startswith('Age')])



- `Diagnosis code`: This code shows the diagnosis from doctors. In the Appendix of bpic11 winner paper by Bose and Aalst, it provides a table for distribution of cases based on diagnosis code combinations. We choose top 8 combinations ({'M13'},{'M16'},{'M11'},{'M14'},{'106'},{'822', '106'}, {'M13', '106'}, {'M13', '822', '106'}) for our experimentation in order to avoid noise and unbalanced splits. 

```bib
@inproceedings{bpic11_winner,
  title={Analysis of patient treatment procedures: The BPI Challenge case study},
  author={R. P. J. C. Bose and W. Aalst},
  year={2011}
}
```

In [ ]:
# function for finding the valid value list

def findValidList(x):
    list_of_values = []
    for index, row in x.iterrows():
        values = []
        for i, v in row.items():
            if pd.notnull(v):
                values.append(v)
        # values = frozenset(values)
        list_of_values.append(values)
    return list_of_values

DiagnosisCode_List = findValidList(df.loc[:, df.columns.str.startswith('Diagnosis code')])


In [ ]:
for index, value in enumerate(DiagnosisCode_List):
    DiagnosisCode_List[index] = list(set(value))
    DiagnosisCode_List[index].sort()

In [ ]:
DiagnosisCode_Unique = [list(y) for y in set([tuple(x) for x in DiagnosisCode_List])]
DiagnosisCode_Unique

In [ ]:
# ['106', '822'] -> ['106']
# ['106', 'M13'] -> ['106']
# ['106', '822', 'M13'] -> ['106']

Changed_Diag_Comb = [['106', '822'], ['106', 'M13'], ['106', '822', 'M13']]

for index, value in enumerate(DiagnosisCode_List):
    if value in Changed_Diag_Comb:
        DiagnosisCode_List[index] = ['106']

- `Treatment code`: Combine the repeated codes in to one list and find a most frquent one.

In [ ]:
# function for finding most frequent element
def mostFrequent(x):
    list_of_values=[]
    for case in x:
        if len(case) == 0:
            list_of_values.append(None)
        else:
            list_of_values.append(max(case, key = case.count))
    return list_of_values

TreatmentCode_List = findValidList(df.loc[:, df.columns.str.startswith('Treatment code')])
TreatmentCode_List = mostFrequent(TreatmentCode_List)

### Add Additinal Features

Add time features, by calculating the time difference between the first event and the last event.

In [ ]:
# Calculate week and day

# total number of traces: 1143 (Hard coding)

Month_list = []
Day_list = []
tbtw_list = []

for case_id in range(1143):
    time_df = df.loc[df['case:concept:name'] == case_id]
    dateStart = datetime.strptime(time_df.iloc[0,-1], '%Y-%m-%dT%H:%M:%S%z')
    dateEnd = datetime.strptime(time_df.iloc[-1,-1], '%Y-%m-%dT%H:%M:%S%z')
    Day_list.append(abs((dateEnd - dateStart).days))
    Month_list.append(abs((dateEnd.year - dateStart.year) * 12 + (dateEnd.month - dateStart.month)))

    for index, row in time_df.iterrows():
        dateNow = datetime.strptime(row['time:timestamp'], '%Y-%m-%dT%H:%M:%S%z')
        tbtw_list.append(abs((dateNow - dateStart).days))



The new dataframe should include these feaures: 

Activity, Department, Number of executions, Activitycode, Producer code, Section, Age, Diagnosis Code, Treatment code and Year.

In [ ]:
processed_df = df[['case:concept:name', 'event:concept:name', 'org:group', 'time:timestamp', 'Activity code', 'Number of executions', 'Producer code', 'Section']]

processed_df = processed_df.assign(Age = Age_List)
processed_df = processed_df.assign(Diagnosis_code = DiagnosisCode_List)
processed_df = processed_df.assign(Treatment_code = TreatmentCode_List)
processed_df = processed_df.assign(tbtw = tbtw_list)

def input_day(row):
    return Day_list[row['case:concept:name']]

def input_month(row):
    return Month_list[row['case:concept:name']]

# case lenth
case_len_list = processed_df.groupby(['case:concept:name']).size().values
def input_cl(row):
    return case_len_list[row['case:concept:name']]

processed_df['Total month'] = processed_df.apply(lambda x: input_month(x), axis=1)
processed_df['Total day'] = processed_df.apply(lambda x: input_day(x), axis=1)

processed_df['Case Lenth'] = processed_df.apply(lambda x: input_cl(x), axis=1)

### Clean nosie in Diagnosis_code and event:concept:name

In [ ]:
# clean Number of executions less than 1

processed_df = processed_df[processed_df['Number of executions'] > 0]

activity_list = processed_df['event:concept:name'].value_counts().loc[lambda x : x>19].index.tolist()
processed_df = processed_df[processed_df['event:concept:name'].isin(activity_list)]

In [ ]:
# clean Treatment_code is null
processed_df = processed_df[~processed_df['Treatment_code'].isnull()]
processed_df.isnull().any()

In [ ]:
processed_df['Diagnosis_code'] = processed_df['Diagnosis_code'].apply(lambda x: ','.join(map(str, x)))
Selected_Diag_Comb = ['M13', 'M16', 'M11', 'M14', '106']
processed_df = processed_df[processed_df['Diagnosis_code'].isin(Selected_Diag_Comb)]



In [ ]:
column_names = ['CaseID', 'Activity', 'Department', 'Timestamps', 'Activity code', 'Number of executions', 'Producer code', 'Section', 'Age', 'Diagnosis code', 'Treatment code', 'tbtw', 'Month', 'Day', 'Case Lenth']
processed_df.columns = column_names

processed_df = processed_df.astype({'Age': 'int64'})

processed_df = processed_df.reset_index(drop=True)



### Save processed dataframe to csv for backup

In [ ]:
processed_df.to_csv(path_or_buf=args['processed_file_name'],sep=';', index=False)

## 2. Understanding the dataset

### Loading the data and parameter files

In [ ]:
log_df = processed_df
log_df.head()

### Data Analysis - Balance of Data Set

In [ ]:
#Checking the Balance of the Dataset, by the target variable
print('Distribution of cases by the target variable - Diagnosis code\n')
print(log_df.groupby(['Diagnosis code'])['CaseID'].nunique())

### Create indexes
Author: Renuka Sindagatta/ Manuel Camargo

Function: creates an index (index encoded set) for a given categorical column.

In [ ]:
def create_index(log_df, column):
    """Creates an idx for a categorical attribute.
    Args:
        log_df: dataframe.
        column: column name.
    Returns:
        index of a categorical attribute pairs.
    """
    temp_list = log_df[[column]].values.tolist()
    subsec_set = {(x[0]) for x in temp_list}
    subsec_set = sorted(list(subsec_set))
    alias = dict()
    if column !='Diagnosis code':
      for i, _ in enumerate(subsec_set):
          alias[subsec_set[i]] = i + 1
    else:
      for i, _ in enumerate(subsec_set):
          alias[subsec_set[i]] = i  
    return alias

create the indexes for the processed dataframe

In [ ]:
# Index creation for activity
# column_names = ['CaseID', 'Activity', 'Department', 'Timestamps', 'Activity code', 'Number of executions', 'Producer code', 'Section', 'Age', 'Diagnosis code', 'Treatment code', 'Month', 'Day']

ac_index = create_index(log_df, 'Activity')
ac_index['start'] = 0
ac_index['end'] = len(ac_index)
index_ac = {v: k for k, v in ac_index.items()}

# Index creation for department/role

rl_index = create_index(log_df, 'Department')
rl_index['start'] = 0
rl_index['end'] = len(rl_index)
index_rl = {v: k for k, v in rl_index.items()}

# Index creation for Diagnosis

di_index = create_index(log_df, 'Diagnosis code')

index_di = {v: k for k, v in di_index.items()}

# Index creation for Treatment
tr_index = create_index(log_df, 'Treatment code')
tr_index['start'] = 0
tr_index['end'] = len(tr_index)
index_tr = {v: k for k, v in tr_index.items()}

#mapping the dictionary values as columns in the dataframe
log_df['ac_index'] = log_df['Activity'].map(ac_index)
log_df['rl_index'] = log_df['Department'].map(rl_index)
log_df['di_index'] = log_df['Diagnosis code'].map(di_index)
log_df['tr_index'] = log_df['Treatment code'].map(tr_index)

print(rl_index)
print(index_rl)
log_df.head()




### Data Analysis - Correlation between features

credits: https://seaborn.pydata.org/examples/many_pairwise_correlations.html

In [ ]:
cor_columns = ['Age','Month', 'tbtw', 'Case Lenth','ac_index','rl_index','tr_index','di_index']

d = log_df[cor_columns]

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Split train test

Author: Renuka Sindagatta/ Manuel Camargo

Function: divides the dataset into test and train sets, preserving traces

In [ ]:
# =============================================================================
# Split an event log dataframe to peform split-validation 
# =============================================================================
def split_train_test(df, percentage):
    cases = df.CaseID.unique()
    num_test_cases = int(np.round(len(cases)*percentage))
    test_cases = cases[:num_test_cases]
    train_cases = cases[num_test_cases:]
    df_train, df_test = pd.DataFrame(), pd.DataFrame()
    for case in train_cases:
        df_train = df_train.append(df[df.CaseID==case]) 
    df_train = df_train.sort_values('Timestamps', ascending=True).reset_index(drop=True)
 
    for case in test_cases:
        df_test = df_test.append(df[df.CaseID==case]) 
    df_test = df_test.sort_values('Timestamps', ascending=True).reset_index(drop=True)
    
    return df_train, df_test 

splitting the dataframe into test and train sets

In [ ]:
# Split validation datasets
log_df_train, log_df_test = split_train_test(log_df, 0.3) # 70%/30%

### normalize_events

Modified from the function of: Renuka Sindagatta/ Manuel Camargo

Function: Normalizes the numerical features

In [ ]:
def normalize_events(log_df,args,features):

#log_df(DataFrame): The dataframe with eventlog data
#args(Dictionary): The set of parameters
#Returns a Dataframe with normalized numerical features
  for feature in features:
    if args['norm_method'] == 'max':
        mean_feature = np.mean(log_df.feature)
        std_feature = np.std(log_df.feature)
        norm = lambda x: (x[feature]-mean_feature)/std_feature
        log_df['%s_norm'%(feature)] = log_df.apply(norm, axis=1)
    elif args['norm_method'] == 'lognorm':
        logit = lambda x: math.log1p(x[feature])
        log_df['%s_log'%(feature)] = log_df.apply(logit, axis=1)
        mean_feature = np.mean(log_df['%s_log'%(feature)])
        std_feature=np.std(log_df['%s_log'%(feature)])
        norm = lambda x: (x['%s_log'%(feature)]-mean_feature)/std_feature
        log_df['%s_norm'%(feature)] = log_df.apply(norm, axis=1)
  return log_df

Adding normalized features: to the training set

In [ ]:
numerical_features = ['Age','Month', 'Day', 'tbtw', 'Case Lenth']
log_df_train = normalize_events(log_df_train,args,numerical_features)
log_df_test = normalize_events(log_df_test,args,numerical_features)
log_df_train.head()

In [ ]:
training_traces = len(log_df_train['CaseID'].unique())
test_traces = len(log_df_test['CaseID'].unique())

print('Number of traces in training set is:'+str(training_traces))
print('Number of traces in test set is:'+str(test_traces))

In [ ]:
#Checking the Balance of the Dataset, by the target variable
print('training dataset')
print(log_df_train.groupby(['Diagnosis code'])['CaseID'].nunique())

print('test dataset')
print(log_df_test.groupby(['Diagnosis code'])['CaseID'].nunique())



### Reformat events

Modified from the function of: Renuka Sindagatta/ Manuel Camargo

Function: converts the dataframe into a dictionary, using the indexes created

In [ ]:
# ==============================================================================
# Reformat events: converts the dataframe into a numerical dictionary
# ==============================================================================

def reformat_events(log_df, ac_index, rl_index,di_index):
    """Creates series of activities, roles and relative times per trace.
    Args:
        log_df: dataframe.
        ac_index (dict): index of activities.
        rl_index (dict): index of roles.
    Returns:
        list: lists of activities, roles and relative times.
    """
    log_df = log_df.to_dict('records')

    temp_data = list()
    log_df = sorted(log_df, key=lambda x: (x['CaseID'], x['Timestamps']))
    for key, group in itertools.groupby(log_df, key=lambda x: x['CaseID']):
        trace = list(group)
        #dynamic features
        ac_order = [x['ac_index'] for x in trace]
        rl_order = [x['rl_index'] for x in trace]
        tbtw = [x['tbtw_norm'] for x in trace]


        #static features: the aggregation used is max(), however, any aggregation could be used since we have a single value for this for the whole trace
        age = max(x['Age_norm'] for x in trace)
        months = max(x['Month_norm'] for x in trace)
        case_len = max(x['Case Lenth_norm'] for x in trace)
        treatment = max(x['tr_index'] for x in trace)

        #outcome
        diagnosis = max(x['di_index'] for x in trace)

        #Adding start and end to the dynamic features
        ac_order.insert(0, ac_index[('start')])
        ac_order.append(ac_index[('end')])
        rl_order.insert(0, rl_index[('start')])
        rl_order.append(rl_index[('end')])
        tbtw.insert(0, 0)
        tbtw.append(0)
        temp_dict = dict(caseid=key,
                         ac_order=ac_order,
                         rl_order=rl_order,
                         tbtw=tbtw,
                         age=age,
                         treatment=treatment,
                         months=months,
                         case_len=case_len,
                         diagnosis=diagnosis)
        temp_data.append(temp_dict)

    return temp_data


converting the training dataframe into a dictionary


In [ ]:
log_train = reformat_events(log_df_train, ac_index, rl_index,di_index)
log_test = reformat_events(log_df_test, ac_index, rl_index,di_index)

In [ ]:
#print a sample of the dictionary
print(log_train[111])

###  Vectorization

Author: Bemali Wickramanayake

Inspired by the code of: Renuka Sindagatta

Function: Creating the Input and Output Tensors 

Notes: Editing needs to finish for this function

In [ ]:
# Support function for Vectirization

# This function returns the maximum trace length (trc_len), and the number of cases for train and test sets (cases)
# The maximum out of trc_len for train and test sets will be used to define the trace length of the dataset that is fed to lstm

def lengths (log):
  trc_len = 1
  cases = 1

  for i,_ in enumerate(log):

    if trc_len <len(log[i]['ac_order']):

        trc_len = len(log[i]['ac_order'])
        cases += 1
    else:
        cases += 1

  return trc_len, cases


In [ ]:
#Obtain the trc_len and cases for each set

trc_len_train, cases_train = lengths(log_train)
trc_len_test, cases_test = lengths(log_test)

trc_len = trc_len_train
if trc_len < trc_len_test:
  trc_len = trc_len_test

print("trace_length: "+str(trc_len)+", training cases: "+str(cases_train)+", test cases: "+str(cases_test))

In [ ]:
def vectorization(log, ac_index, rl_index, di_index, tr_index, trc_len,cases):

#Example function with types documented in the docstring.
#Args:
        #log: event log data in a dictionary.
        #ac_index (dict): index of activities.
        #rl_index (dict): index of roles (departments).
        #di_index (dict) : index of diagnosis codes.

#Returns:vec: Dictionary that contains all the LSTM inputs. """

  vec = {'prefixes':dict(), 'static':dict(),'diagnosis':[]} 
  len_ac = trc_len  

  for i ,_ in enumerate(log):
  
    padding = np.zeros(len_ac-len(log[i]['ac_order']))

    if i == 0:
            vec['prefixes']['x_ac_inp'] = np.array(np.append(log[i]['ac_order'],padding))
            vec['prefixes']['x_rl_inp'] = np.array(np.append(log[i]['rl_order'],padding))
            vec['prefixes']['xt_inp'] = np.array(np.append(log[i]['tbtw'],padding))
            vec['static']['x_age_inp'] = np.array(log[i]['age'])
            vec['static']['x_months_inp'] = np.array(log[i]['months'])
            vec['static']['x_cl_inp'] = np.array(log[i]['case_len'])
            vec['static']['x_tr_inp'] = np.array(log[i]['treatment'])
            vec['diagnosis'] = np.array(log[i]['diagnosis'])


            #print(len(vec['prefixes']['x_ac_inp']))

  
    vec['prefixes']['x_ac_inp'] = np.concatenate((vec['prefixes']['x_ac_inp'],
                                                          np.array(np.append(log[i]['ac_order'],padding))), axis=0)
    vec['prefixes']['x_rl_inp'] = np.concatenate((vec['prefixes']['x_rl_inp'],
                                                          np.array(np.append(log[i]['rl_order'],padding))), axis=0)
    vec['prefixes']['xt_inp'] = np.concatenate((vec['prefixes']['xt_inp'],
                                                        np.array(np.append(log[i]['tbtw'],padding))), axis=0)
    vec['static']['x_age_inp'] = np.append(vec['static']['x_age_inp'],log[i]['age'])
    vec['static']['x_months_inp'] = np.append(vec['static']['x_months_inp'],log[i]['months'])
    vec['static']['x_cl_inp'] = np.append(vec['static']['x_cl_inp'],log[i]['case_len'])
    vec['static']['x_tr_inp'] = np.append(vec['static']['x_tr_inp'],log[i]['treatment'])
    vec['diagnosis'] = np.append(vec['diagnosis'],log[i]['diagnosis'])
  

  

  
  #The concatenation returns a flattened vector. Hence, reshaping the vectors at the end
  vec['prefixes']['x_ac_inp'] = np.reshape(vec['prefixes']['x_ac_inp'],(cases,len_ac))
  vec['prefixes']['x_rl_inp'] = np.reshape(vec['prefixes']['x_rl_inp'],(cases,len_ac))
  vec['prefixes']['xt_inp'] = np.reshape(vec['prefixes']['xt_inp'],(cases,len_ac))

  #one-hot-encoding the y class
  vec['diagnosis'] = ku.to_categorical(vec['diagnosis'],
                                               num_classes=len(di_index))

  return vec


converting the training log (dictionary) into a Tensor

In [ ]:
vec_train = vectorization(log_train,ac_index,rl_index,di_index,tr_index,trc_len,cases_train)
vec_test = vectorization(log_test,ac_index,rl_index,di_index,tr_index,trc_len,cases_test)

In [ ]:
vec_train['diagnosis']

### Initial Embedding weights for the Embedding layer

In [ ]:
# Load embedded matrix
ac_weights = ku.to_categorical(sorted(index_ac.keys()), len(ac_index))
#print('AC_WEIGHTS', ac_weights)
rl_weights =  ku.to_categorical(sorted(index_rl.keys()), len(rl_index))
#print('RL_WEIGHTS', rl_weights)
tr_weights =  ku.to_categorical(sorted(index_tr.keys()), len(tr_index))


### Saving the Processed Tensor and Other Support Data

In [ ]:
# args['processed_training_vec'] = vec_train
# args['processed_test_vec'] = vec_test


# # converting the weights into a dictionary and saving
# weights = {'ac_weights':ac_weights, 'rl_weights':rl_weights, 'diagnoses':len(di_index)}
# args['weights'] = weights

# indexes = {'index_ac':index_ac, 'index_rl':index_rl, 'index_di':index_di, 'index_tr':index_tr}
# args['indexes'] = indexes


In [ ]:

# saving the processed tensor
with open(args['processed_training_vec'], 'wb') as fp:
    pickle.dump(vec_train, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open(args['processed_test_vec'], 'wb') as fp:
    pickle.dump(vec_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

# converting the weights into a dictionary and saving
weights = {'ac_weights':ac_weights, 'rl_weights':rl_weights, 'diagnoses':len(di_index), 'tr_weights':tr_weights}
with open(args['weights'], 'wb') as fp:
    pickle.dump(weights, fp, protocol=pickle.HIGHEST_PROTOCOL)

# converting the weights into a dictionary and saving
indexes = {'index_ac':index_ac, 'index_rl':index_rl, 'index_di':index_di, 'index_tr':index_tr}
with open(args['indexes'], 'wb') as fp:
    pickle.dump(indexes, fp, protocol=pickle.HIGHEST_PROTOCOL)

#saving the arguements (args)
with open(args['args'], 'wb') as fp:
    pickle.dump(args, fp, protocol=pickle.HIGHEST_PROTOCOL)




## 3. Dynamic Features with Attention

### Loading the data and parameter files

In [ ]:
# Open from local file

with open(os.path.join(OUTPUT_DIR, 'args.p'), 'rb') as fp:
    args = pickle.load(fp)

with open(args['processed_training_vec'], 'rb') as fp:
    vec_train = pickle.load(fp)
with open(args['processed_test_vec'], 'rb') as fp:
    vec_test = pickle.load(fp)
    
with open(args['weights'], 'rb') as fp:
    weights = pickle.load(fp)
ac_weights = weights['ac_weights']
rl_weights = weights['rl_weights']
tr_weights = weights['tr_weights']
diagnoses = weights['diagnoses']

with open(args['indexes'], 'rb') as fp:
    indexes = pickle.load(fp)
index_ac = indexes['index_ac']
index_rl = indexes['index_rl']
index_di = indexes['index_di']
index_tr = indexes['index_tr']
index_di

In [ ]:
# # read from previous
# vec_train = args['processed_training_vec']
# vec_test = args['processed_test_vec']

# weights = args['weights']
# ac_weights = weights['ac_weights']
# rl_weights = weights['rl_weights']
# diagnoses = weights['diagnoses']
# tr_weights = weights['tr_weights']

# indexes = args['indexes']
# index_ac = indexes['index_ac']
# index_rl = indexes['index_rl']

# index_di = indexes['index_di']
# index_di

### Build Model

Author: Bemali Wickramanayake

Model Details: This model computes the attention weights for the dynamic 'prefix' features and uses them in the final outcome prediction together with the static features

In [ ]:
import keras.layers as L
from keras import backend as K
from keras.layers import Embedding

from keras.layers import Lambda, dot, Activation, concatenate, Dense


#Initializing variables
vec = vec_train
output_folder = MY_WORKSPACE_DIR

MAX_LEN = args['n_size']
dropout_input = 0.15
dropout_context=0.15
  # number of lstm cells
incl_time = True 
incl_res = True
lstm_size_alpha=args['l_size'] #number of lstm cells
lstm_size_beta=args['l_size']
print("Training prefix-attention model")

l2reg=0.0001

output_length = diagnoses

 
  #Inputs include activity, resource and time - time is normalised- 0 mean and unit variance

#Dynamic Inputs
ac_input = Input(shape=(vec['prefixes']['x_ac_inp'].shape[1], ), name='ac_input') 
rl_input = Input(shape=(vec['prefixes']['x_rl_inp'].shape[1], ), name='rl_input')
t_input = Input(shape=(vec['prefixes']['xt_inp'].shape[1], 1), name='t_input')

#static inputs
# age_input = Input(shape = (1, ),name = 'age_input') #cl_input_d #vec['static']['x_age_inp']
# cl_input_d = Input(shape =(1, ),name = 'cl_input_d') #case length (the duration of a case) in days #vec['static']['x_cl_inp']
# cl_input_y = Input(shape=(1, ),name = 'cl_input_y') #case length (the duration of a case) in years, rounded down #vec['static']['x_years_inp']

#Activity Embedding - dynamic input 1
  
ac_embs = L.Embedding(ac_weights.shape[0],
                            ac_weights.shape[1],
                            weights=[ac_weights], #the one hot encoded activity weight matrix used as the initial weight matrix
                            input_length=vec['prefixes']['x_ac_inp'].shape[1],
                            trainable=True, name='ac_embedding')(ac_input)

dim_ac =ac_weights.shape[1]   



#Role Embedding - dynamic input 2
rl_embs = Embedding(rl_weights.shape[0],
                            rl_weights.shape[1],
                            weights=[rl_weights],
                            input_length=vec['prefixes']['x_rl_inp'].shape[1],
                            trainable=True, name='rl_embedding')(rl_input)

dim_rl = rl_weights.shape[1]
      

#Time input

time_embs=t_input
dim_t = 1

#Concatenated Input Vector

full_embs = L.concatenate([ac_embs,rl_embs,time_embs],name = 'full_embs')
full_embs = L.Dropout(dropout_input)(full_embs)

#Set up the LSTM networks

#LSTM 
alpha = L.Bidirectional(L.CuDNNLSTM(lstm_size_alpha, return_sequences=True),
                                    name='alpha')
beta = L.Bidirectional(L.LSTM(lstm_size_beta, return_sequences=True),
                                   name='beta')


#Dense layer for attention
alpha_dense = L.Dense(1, kernel_regularizer=l2(l2reg))
beta_dense = L.Dense(1,
                             activation='tanh', kernel_regularizer=l2(l2reg))

#Compute alpha, timestep attention

alpha_out = alpha(full_embs)
alpha_out = L.TimeDistributed(alpha_dense, name='alpha_dense')(alpha_out)
alpha_out = L.Softmax(name='timestep_attention', axis=1)(alpha_out)

#Compute beta, feature attention
beta_out = beta(full_embs)
beta_out = L.TimeDistributed(beta_dense, name='feature_attention')(beta_out)



  
#Compute context vector based on attentions and embeddings
c_t = L.Multiply()([alpha_out, beta_out,full_embs])
c_t = L.Lambda(lambda x: K.sum(x, axis=1))(c_t)

  
#contexts = L.concatenate([c_t,age_input,cl_input_d], name='contexts')
contexts = L.Dropout(dropout_context)(c_t)


  
act_output = Dense(output_length,
                       activation='softmax',
                       kernel_initializer='glorot_uniform',
                       name='act_output')(contexts)


#model = Model(inputs=[ac_input, rl_input, t_input,age_input,cl_input_d], outputs=act_output)
model1 = Model(inputs=[ac_input, rl_input, t_input], outputs=act_output)

tf.keras.utils.plot_model(
    model1,
    to_file="model1.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)



### Compile Model

In [ ]:
# Adam

args['optim']='Adam'

Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999,
                   epsilon=None, decay=0.0, amsgrad=False,name = 'Adam')
model1.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    
model1.summary()

### Fit Model

In [ ]:

    
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
#
#    # Output file

output_file_path = os.path.join(output_folder,
                                    'models/model_rd_' + str(args['n_size']) +
                                    ' ' + args['optim'] + '_'  + args['log_name']  +
                                    '_{epoch:02d}-{val_loss:.2f}.h5')
# print('This is the output file path ', output_file_path)

    # Saving
model_checkpoint = ModelCheckpoint(output_file_path,
                                       monitor='val_loss',
                                       verbose=1,
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.5,
                                   patience=10,
                                   verbose=0,
                                   mode='auto',
                                   min_delta=0.0001,
                                   cooldown=0,
                                   min_lr=0)

model_inputs = [vec['prefixes']['x_ac_inp']]
model_inputs.append(vec['prefixes']['x_rl_inp'])
model_inputs.append(vec['prefixes']['xt_inp'])
#model_inputs.append(vec['static']['x_age_inp'])
#model_inputs.append(vec['static']['x_cl_inp'])



with tf.compat.v1.Session() as sess:
     sess.run(tf.compat.v1.global_variables_initializer())

history1 = model1.fit(model_inputs,
            {'act_output':vec['diagnosis']},
            validation_split=0.15,
            verbose=1,
            callbacks=[early_stopping, model_checkpoint,lr_reducer],
            # callbacks=[early_stopping,lr_reducer],
            batch_size=64,
            epochs=100)
#return model

### Plot accuracy and loss graph

In [ ]:
# list all data in history
print(history1.history.keys())
# summarize history for accuracy
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy (Dynamic)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss (Dynamic)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Evaluating the Prediction performance of Diagnosis for new samples

In [ ]:
# Generating Inputs

x_test = [vec_test['prefixes']['x_ac_inp']]
x_test.append(vec_test['prefixes']['x_rl_inp'])
x_test.append(vec_test['prefixes']['xt_inp'])
#x_test.append(vec_test['static']['x_age_inp'])
#x_test.append(vec_test['static']['x_cl_inp'])


y_test = vec_test['diagnosis']

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model1.evaluate(x_test, y_test, batch_size=10)# fix the padding size to be common for both test and train
print("test loss, test acc:", results)

#Prediction

y_pred = model1.predict(x_test)


### Confusion Matrix

In [ ]:
#Confusion Matrix

import seaborn as sns


matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)
df_cm = pd.DataFrame(matrix, index = [index_di[i] for i in range(5)],
                  columns = [index_di[i] for i in range(5)])


plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

## 4. Dynamic and Static Features with Attention

### Loading the data and parameter files

In [ ]:
# Open from local file

with open(os.path.join(OUTPUT_DIR, 'args.p'), 'rb') as fp:
    args = pickle.load(fp)

with open(args['processed_training_vec'], 'rb') as fp:
    vec_train = pickle.load(fp)
with open(args['processed_test_vec'], 'rb') as fp:
    vec_test = pickle.load(fp)
    
with open(args['weights'], 'rb') as fp:
    weights = pickle.load(fp)
ac_weights = weights['ac_weights']
rl_weights = weights['rl_weights']
tr_weights = weights['tr_weights']
diagnoses = weights['diagnoses']

with open(args['indexes'], 'rb') as fp:
    indexes = pickle.load(fp)
index_ac = indexes['index_ac']
index_rl = indexes['index_rl']
index_di = indexes['index_di']
index_tr = indexes['index_tr']
index_di

In [ ]:
# # read from previous
# vec_train = args['processed_training_vec']
# vec_test = args['processed_test_vec']

# weights = args['weights']
# ac_weights = weights['ac_weights']
# rl_weights = weights['rl_weights']
# diagnoses = weights['diagnoses']

# indexes = args['indexes']
# index_ac = indexes['index_ac']
# index_rl = indexes['index_rl']

# index_di = indexes['index_di']
# index_di

### Build Model

In [ ]:
import keras.layers as L
from keras import backend as K
from keras.layers import Embedding

from keras.layers import Lambda, dot, Activation, concatenate, Dense


#Initializing variables
vec = vec_train
output_folder = MY_WORKSPACE_DIR

MAX_LEN = args['n_size']
dropout_input = 0.15
dropout_context=0.15
  # number of lstm cells
incl_time = True 
incl_res = True
lstm_size_alpha=args['l_size'] #number of lstm cells
lstm_size_beta=args['l_size']
print("Training prefix-attention model")

l2reg=0.0001

output_length = diagnoses

 
  #Inputs include activity, resource and time - time is normalised- 0 mean and unit variance

#Dynamic Inputs
ac_input = Input(shape=(vec['prefixes']['x_ac_inp'].shape[1], ), name='ac_input') 
rl_input = Input(shape=(vec['prefixes']['x_rl_inp'].shape[1], ), name='rl_input')
t_input = Input(shape=(vec['prefixes']['xt_inp'].shape[1], 1), name='t_input')

#static inputs
age_input = Input(shape = (1, ),name = 'age_input') #cl_input_d #vec['static']['x_age_inp']
# cl_input_d = Input(shape =(1, ),name = 'cl_input_d') #case length (the duration of a case) in days #vec['static']['x_cl_inp']
# cl_input_y = Input(shape=(1, ),name = 'cl_input_y') #case length (the duration of a case) in years, rounded down #vec['static']['x_years_inp']
tr_input = Input(shape=(1, ),name = 'tr_input')


#Activity Embedding - dynamic input 1
  
ac_embs = L.Embedding(ac_weights.shape[0],
                            ac_weights.shape[1],
                            weights=[ac_weights], #the one hot encoded activity weight matrix used as the initial weight matrix
                            input_length=vec['prefixes']['x_ac_inp'].shape[1],
                            trainable=True, name='ac_embedding')(ac_input)

dim_ac =ac_weights.shape[1]   



#Role Embedding - dynamic input 2
rl_embs = Embedding(rl_weights.shape[0],
                            rl_weights.shape[1],
                            weights=[rl_weights],
                            input_length=vec['prefixes']['x_rl_inp'].shape[1],
                            trainable=True, name='rl_embedding')(rl_input)

dim_rl = rl_weights.shape[1]


#Time input

time_embs=t_input
dim_t = 1

#Concatenated Input Vector

full_embs = L.concatenate([ac_embs,rl_embs,time_embs],name = 'full_embs')
full_embs = L.Dropout(dropout_input)(full_embs)

#Set up the LSTM networks

#LSTM 
alpha = L.Bidirectional(L.CuDNNLSTM(lstm_size_alpha, return_sequences=True),
                                    name='alpha')
beta = L.Bidirectional(L.LSTM(lstm_size_beta, return_sequences=True),
                                   name='beta')


#Dense layer for attention
alpha_dense = L.Dense(1, kernel_regularizer=l2(l2reg))
beta_dense = L.Dense(1,activation='tanh', kernel_regularizer=l2(l2reg))

#Compute alpha, timestep attention

alpha_out = alpha(full_embs)
alpha_out = L.TimeDistributed(alpha_dense, name='alpha_dense')(alpha_out)
alpha_out = L.Softmax(name='timestep_attention', axis=1)(alpha_out)

#Compute beta, feature attention
beta_out = beta(full_embs)
beta_out = L.TimeDistributed(beta_dense, name='feature_attention')(beta_out)

  
#Compute context vector based on attentions and embeddings
c_t = L.Multiply()([alpha_out, beta_out,full_embs])
c_t = L.Lambda(lambda x: K.sum(x, axis=1))(c_t)


# Static Features with attention

#Treatment Embedding - static input
# tr_embs = L.Embedding(tr_weights.shape[0],
#                             tr_weights.shape[1],
#                             weights=[tr_weights], #the one hot encoded activity weight matrix used as the initial weight matrix
#                             input_length=len(vec['static']['x_tr_inp']),
#                             trainable=True, name='tr_embedding')(tr_input)

# dim_tr =tr_weights.shape[1]   

#Dense layer for attention of static features

alpha_dense_age = L.Dense(1, kernel_regularizer=l2(l2reg),name = 'alpha_dense_age')
# alpha_dense_cl = L.Dense(1, kernel_regularizer=l2(l2reg),name = 'alpha_dense_cl')
alpha_dense_tr = L.Dense(1, kernel_regularizer=l2(l2reg),name = 'alpha_dense_tr')

#Compute static attentions

age_out = alpha_dense_age(age_input)
age_out = L.Softmax(name='age_static_attention', axis=1)(age_out)

# cl_out = alpha_dense_cl(age_input)
# cl_out = L.Softmax(name='cl_static_attention', axis=1)(cl_input_d)

tr_out = alpha_dense_tr(tr_input)
tr_out = L.Softmax(name='tr_static_attention', axis=1)(tr_out)


  
contexts = L.concatenate([c_t,age_out,tr_out], name='contexts')
contexts = L.Dropout(dropout_context)(contexts)


  
act_output = Dense(output_length,
                       activation='softmax',
                       kernel_initializer='glorot_uniform',
                       name='act_output')(contexts)

 #changing the optimizer
 #----- delete this bit if works--------
#args['optim'] = 'Adagrad'
 #------------------------------------

model2 = Model(inputs=[ac_input, rl_input, t_input,age_input,tr_input], outputs=act_output)
#model = Model(inputs=[ac_input, rl_input, t_input], outputs=act_output)

tf.keras.utils.plot_model(
    model2,
    to_file="model2.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)



### Compile model

In [ ]:
args['optim']='Adam'

Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999,
                   epsilon=None, decay=0.0, amsgrad=False,name = 'Adam')
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()

### Fit model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
#
#    # Output file
output_file_path = os.path.join(output_folder,
                                    'models/model_rd_' + str(args['n_size']) +
                                    ' ' + args['optim'] + '_'  + args['log_name']  +
                                    '_{epoch:02d}-{val_loss:.2f}.h5')
# print('This is the output file path ', output_file_path)
    # Saving
model_checkpoint = ModelCheckpoint(output_file_path,
                                       monitor='val_loss',
                                       verbose=1,
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.5,
                                   patience=10,
                                   verbose=0,
                                   mode='auto',
                                   min_delta=0.0001,
                                   cooldown=0,
                                   min_lr=0)

model_inputs = [vec['prefixes']['x_ac_inp']]
model_inputs.append(vec['prefixes']['x_rl_inp'])
model_inputs.append(vec['prefixes']['xt_inp'])
model_inputs.append(vec['static']['x_age_inp'])
model_inputs.append(vec['static']['x_cl_inp'])



with tf.compat.v1.Session() as sess:
     sess.run(tf.compat.v1.global_variables_initializer())

history2 = model2.fit(model_inputs,
            {'act_output':vec['diagnosis']},
            validation_split=0.15,
            verbose=1,
            callbacks=[early_stopping, model_checkpoint,lr_reducer],
            # callbacks=[early_stopping,lr_reducer],
            batch_size=64,
            epochs=100)
#return model

### Plot accuracy and loss graph

In [ ]:
# list all data in history
print(history2.history.keys())
# summarize history for accuracy
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy (Dynamic and Static)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss (Dynamic and Static)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Evaluating the Prediction performance of Diagnosis for new samples

In [ ]:
# Generating Inputs

x_test = [vec_test['prefixes']['x_ac_inp']]
x_test.append(vec_test['prefixes']['x_rl_inp'])
x_test.append(vec_test['prefixes']['xt_inp'])
x_test.append(vec_test['static']['x_age_inp'])
x_test.append(vec_test['static']['x_cl_inp'])


y_test = vec_test['diagnosis']

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model2.evaluate(x_test, y_test, batch_size=10)# fix the padding size to be common for both test and train
print("test loss, test acc:", results)

#Prediction

y_pred = model2.predict(x_test)


### Confusion Matrix

In [ ]:
#Confusion Matrix

import seaborn as sn


matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)
df_cm = pd.DataFrame(matrix, index = [index_di[i] for i in range(5)],
                  columns = [index_di[i] for i in range(5)])

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

### Visualizing Attention Weights

credits: https://stackoverflow.com/questions/53867351/how-to-visualize-attention-weights